**Выполнил с двух блоков с news.mail.ru и наполнил базу данных MongoDB**

In [1]:
import requests
from pprint import pprint

In [2]:
from lxml import html
from pymongo import MongoClient

In [3]:
!apt install mongodb >log
!service mongodb start



 * Starting database mongodb
   ...done.


In [4]:
client = MongoClient('127.0.0.1', 27017)

In [5]:
client.status

Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'status')

In [6]:
client.list_database_names()

['admin', 'local']

In [7]:
db = client.news

In [8]:
db.create_collection('news_list')

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'news'), 'news_list')

In [9]:
list (db.list_collections()) 

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'news.news_list',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': UUID('7a2d1ecf-914e-44ef-a8de-ee97f6d4bf84')},
  'name': 'news_list',
  'options': {},
  'type': 'collection'}]

In [10]:
collection = db.news_list

In [11]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.34'}

In [12]:
response = requests.get('https://news.mail.ru/', headers=header)

In [13]:
dom = html.fromstring(response.text)

In [15]:
#новости с фото
news_photo = dom.xpath("//div[@class='daynews__item' or @class='daynews__item daynews__item_big' or @class='daynews__item hidden_small']")

#новости со списка
news_list = dom.xpath("//li[@class = 'list__item hidden_medium hidden_large' or @class = 'list__item']")

Кажется что-то перемудрил заставляя каждый раз делать отдельный запрос на сайт чтоб найти информацию. Так же на одном из сайтов столкнулся с проблемой что класс называется по другому. По этому ввел оператор "или".

In [17]:
list_all_news = [] 

for item in news_photo:
  news_data = {}
  news = item.xpath(".//span[@class='photo__title photo__title_new photo__title_new_hidden js-topnews__notification']/text()")[0].replace('\xa0', ' ')
  link = item.xpath(".//a/@href")[0]
  response_news = requests.get(link, headers=header)
  dom_news = html.fromstring(response_news.text)
  data = dom_news.xpath("//span[@datetime]/@datetime")[0]
  source = dom_news.xpath("//a[@class='link color_gray breadcrumbs__link']//text()")[0]
  
  news_data['News'] = news
  news_data['Link'] = link
  news_data['Source'] = source
  news_data['Data'] = data

  list_all_news.append(news_data)

Волевая, сильная, непоколебимая: не стало Нины Ургант
https://news.mail.ru/society/49067454/
2021-12-03T19:45:36+03:00
Известия
Первый российский регион достиг целевого уровня вакцинации
https://news.mail.ru/society/49064626/
2021-12-03T15:18:33+03:00
РБК
«Омикрон» за неделю охватил 26 стран. Первые результаты
https://news.mail.ru/incident/49064802/
2021-12-03T16:09:49+03:00
Телеканал 360°
Талибы издали указ о правах женщин
https://news.mail.ru/politics/49062721/
2021-12-03T13:23:30+03:00
Коммерсантъ
Олимпийский чемпион назвал комфортную сумму для жизни в месяц
https://sportmail.ru/news/boxing/49057264/
2021-12-03T09:42:23+03:00
РБК


In [18]:
for item in news_list:
  news_data = {}
  tmp_news = item.xpath(".//a/text()")
  if len(tmp_news)>0:
    news = tmp_news[0].replace('\xa0', ' ')
    link = item.xpath(".//a/@href")[0]
    response_news = requests.get(link, headers=header)
    dom_news = html.fromstring(response_news.text)
    data = dom_news.xpath("//span[@datetime]/@datetime")[0]
    source = dom_news.xpath("//a[@class='link color_gray breadcrumbs__link']//text()")[0]
    news_data['News'] = news
    news_data['Link'] = link
    news_data['Source'] = source
    news_data['Data'] = data

    list_all_news.append(news_data)

In [19]:
pprint(list_all_news)

[{'Data': '2021-12-03T13:23:30+03:00',
  'Link': 'https://news.mail.ru/politics/49062721/',
  'News': 'Талибы издали указ о правах женщин',
  'Source': 'Коммерсантъ'},
 {'Data': '2021-12-03T09:42:23+03:00',
  'Link': 'https://sportmail.ru/news/boxing/49057264/',
  'News': 'Олимпийский чемпион назвал комфортную сумму для жизни в месяц',
  'Source': 'РБК'},
 {'Data': '2021-12-03T11:07:59+03:00',
  'Link': 'https://news.mail.ru/politics/49058713/',
  'News': 'Ангелу Меркель проводили с факельным шествием (фото)',
  'Source': 'Коммерсантъ'},
 {'Data': '2021-12-03T12:03:45+03:00',
  'Link': 'https://news.mail.ru/society/49053293/',
  'News': 'Главную елку Британии высмеяли в соцсетях',
  'Source': 'BBC News Русская служба'},
 {'Data': '2021-12-03T09:30:29+03:00',
  'Link': 'https://sportmail.ru/news/figure-skating/49054837/',
  'News': 'Фигуристам поднимают возрастной ценз. Чем это грозит для России',
  'Source': 'РБК'},
 {'Data': '2021-12-03T03:49:42+03:00',
  'Link': 'https://news.mail.ru

In [ ]:
for news in list_all_news:
  collection.insert_one(news)

In [ ]:
list(collection.find())

[{'Data': '2021-12-01',
  'Link': 'https://news.mail.ru/politics/49033287/',
  'News': 'Нуланд пригрозила России новыми санкциями за Украину',
  'Source': '© РИА Новости',
  '_id': ObjectId('61a7c752b15d5345b74afc45')},
 {'Data': '2021-12-01',
  'Link': 'https://sportmail.ru/news/sync-swimming/49030896/',
  'News': 'Трехкратная чемпионка Игр завершила карьеру ради работы в Кремле',
  'Source': 'РБК',
  '_id': ObjectId('61a7c752b15d5345b74afc46')},
 {'Data': '2021-12-01',
  'Link': 'https://news.mail.ru/society/49024348/',
  'News': 'В Москве простились с Александром Градским (фото)',
  'Source': 'Коммерсантъ',
  '_id': ObjectId('61a7c752b15d5345b74afc47')},
 {'Data': '2021-12-01',
  'Link': 'https://news.mail.ru/economics/49035403/',
  'News': 'Размер средней зарплаты в России превысил 54 000 рублей',
  'Source': 'Ведомости',
  '_id': ObjectId('61a7c752b15d5345b74afc48')},
 {'Data': '2021-12-01',
  'Link': 'https://news.mail.ru/society/49034720/',
  'News': 'В Китае заявили об открытии